In [1]:
import numpy as np # linear algebra
import tensorflow as tf

In [2]:
mygraph = tf.Graph() # 创建一个名为mygraph的计算图，可以不创建使用默认的图

In [3]:
with mygraph.as_default(): # 将mygraph计算图设置为当前使用的图。同一个graph下不同命名空间的变量可以相互调用
    with tf.name_scope('variables'): 
        global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')
        total_outputs = tf.Variable(0.0, dtype=tf.float32, trainable=False, name='total_outputs')
        # 创建两个 tf.Variable() 用于存储可变数值，因为全局步数和总输出需要根据程序运行，增加相应数值。需要initial Variable变量
    with tf.name_scope('transformation'):
        with tf.name_scope('input'):
            a = tf.placeholder(tf.float32, shape=(None), name='input_placeholder_a') 
            # 输入数值一般使用占位符，以免大量数据保存在graph中，减慢加载速度
        with tf.name_scope('middle'):
            b = tf.reduce_prod(a, name='reduce_prod_b') # 全部元素相乘
            c = tf.reduce_sum(a, name='reduce_sum_c') # 全部元素相加
        with tf.name_scope('output'):
            output = tf.add(b,c,name='output')
    with tf.name_scope('update'):
        update_total = total_outputs.assign_add(output) # assign_add()累加，i += 1
        total_step = global_step.assign_add(1)
        
    with tf.name_scope('summaries'):
        average = tf.div(update_total, tf.cast(total_step, tf.float32), name='average') #tf.cast()转换数据类型
        tf.summary.scalar('output_summary', output)
        tf.summary.scalar('total_output_summary', update_total)
        tf.summary.scalar('average_summary', average)
        # tf.summary.scalar() Outputs a `Summary` protocol buffer containing a single scalar value.

    with tf.name_scope('global_operations'):
        init = tf.global_variables_initializer() # 初始化全部 tf.Variable()变量
        merged_summaries = tf.summary.merge_all() # 将上面所有的summary合并在一起
        

In [4]:
sess = tf.Session(graph=mygraph) # 运行绘画，指定要运行哪一个计算图
summary_writer = tf.summary.FileWriter('./mygraph', sess.graph)

In [5]:
sess.run(init)

In [6]:
def run_graph(input_tensor):
    input_dict = {a: input_tensor}
    outputs, step, summary = sess.run([output, global_step, merged_summaries], feed_dict=input_dict)
    print('outputs: ',outputs)
    print('step: ', step)
    summary_writer.add_summary(summary, global_step=step)
    # summary_writer.add_summary(summary, global_step=None)
    # Adds a `Summary` protocol buffer to the event file.
    summary_writer.add_graph(sess.graph)
    # summary_writer.add_graph(graph, global_step=None, graph_def=None)
    # Adds a `Graph` to the event file.

In [7]:
run_graph([2,8])
run_graph([3,1,1])
run_graph([5])
run_graph([2,5,9,9])
run_graph([1,0,0])
run_graph([[1],[0],[0]])

outputs:  26.0
step:  1
outputs:  8.0
step:  2
outputs:  10.0
step:  3
outputs:  835.0
step:  4
outputs:  1.0
step:  5
outputs:  1.0
step:  6


In [8]:
summary_writer.close()
sess.close()

In [9]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
LOG_DIR = './mygraph' # Here you have to put your log directory
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2018-11-20 02:15:55--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.72.245.79, 52.73.9.93, 52.71.139.107, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.72.245.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  --.-KB/s    in 0.1s    

2018-11-20 02:15:55 (44.1 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
https://3f1cbbc2.ngrok.io


1. Tensorboard - Scalar
![](https://i.loli.net/2018/11/20/5bf36fe62c9e8.png)

2. Tensorboard - Graph
![](https://i.loli.net/2018/11/20/5bf36fe66c7a2.png)
![](https://i.loli.net/2018/11/20/5bf370b153c76.png)

Reference：Wanmen 36.12